<a href="https://colab.research.google.com/github/DojunPark/Machine_Translation/blob/master/08_bleu_function_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Oct 24 02:40:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 13.7 gigabytes of available RAM



# installing the tokenizers

In [3]:
import time
start = time.time()

!pip install konlpy
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

print(f'mecab 설치 소요 시간: {((time.time()-start)/60):.2f} min')

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 14.0MB/s 
     |████████████████████████████████| 460kB 54.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.10).
git is already the newest version (1:2.17.1-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Installing automake (A dependency for mecab-ko)
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/re

In [4]:
start = time.time()

!python -m spacy download en

print(f'spacy tokenizer(en) 설치 소요 시간: {((time.time()-start)/60):.2f} min')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
spacy tokenizer(en) 설치 소요 시간: 0.08 min


In [5]:
from konlpy.tag import Mecab
import spacy

mecab = Mecab()
spacy_en = spacy.load('en')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

print('tokenization test with sample texts')
print('tokenizing for Korean with mecab: ', mecab.morphs('안녕하세요 저는 건국대학교에 재학 중인 박도준입니다.'))
print('tokenizing for English: ', tokenize_en('Hello, I am Dojun Park, a student at Konkuk University.'))

tokenization test with sample texts
tokenizing for Korean with mecab:  ['안녕', '하', '세요', '저', '는', '건국대', '학교', '에', '재학', '중', '인', '박도준', '입니다', '.']
tokenizing for English:  ['Hello', ',', 'I', 'am', 'Dojun', 'Park', ',', 'a', 'student', 'at', 'Konkuk', 'University', '.']


# preparing the data set

In [6]:
!pip uninstall torchtext -y

Uninstalling torchtext-0.3.1:
  Successfully uninstalled torchtext-0.3.1


In [7]:
!pip install torchtext

     |████████████████████████████████| 4.5MB 4.8MB/s 
     |████████████████████████████████| 1.1MB 59.1MB/s 


In [8]:
from torchtext.data import Field, TabularDataset, BucketIterator

korean = Field(tokenize=mecab.morphs, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>')

fields = {'kor': ('src', korean), 'eng':('trg', english)}

train_data, valid_data, test_data = TabularDataset.splits(
                                                    path = '/content/drive/My Drive/Colab Notebooks',
                                                    train = 'train2.csv',
                                                    validation = 'valid2.csv',
                                                    test = 'test2.csv',
                                                    format = 'csv',
                                                    fields = fields)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes f

In [9]:
korean.build_vocab(train_data, min_freq=2)
english.build_vocab(train_data, min_freq=2)

In [10]:
print(train_data[0].__dict__)

{'src': ['영화', '가', '관객', '의', '압도', '적', '인', '지지', '를', '받', '으며', '덩달', '아', '퀸', '신드롬', '까지', '일으켰', '다', '.'], 'trg': ['the', 'movie', 'even', 'created', 'queen', 'syndrome', 'with', 'overwhelming', 'support', 'from', 'the', 'audience', '.']}


In [11]:
print(valid_data[0].__dict__)

{'src': ['중국', '이', '미국', '산', '대두', ',', '보잉', '사', ',', '자동차', ',', '반도체', ',', '휴대폰', '을', '언급', '한', '이유', '다', '.'], 'trg': ['that', "'s", 'why', 'china', 'mentioned', 'u.s.', 'soybeans', ',', 'boeing', ',', 'automobiles', ',', 'semiconductors', 'and', 'cell', 'phones', '.']}


In [12]:
print(test_data[0].__dict__)

{'src': ['중', '장기', '적', '으로', 'cj', 'enm', '의', '기업', '가치', '가', '상승', '할', '것', '이', '란', '전망', '을', '내놓', '고', '있', '다', '.'], 'trg': ['it', 'is', 'predicting', 'that', 'cj', 'enm', "'s", 'corporate', 'value', 'will', 'rise', 'in', 'the', 'mid-', 'to', 'long', '-', 'term', '.']}


# Modelling

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_length,
        device):

        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout)
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        # src shape: (src_len, N)
        src_mask = src.transpose(0, 1) == self.src_pad_idx
        # (N, src_len)
        return src_mask

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length).unsqueeze(1).expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )

        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device)

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask = src_padding_mask,
            tgt_mask = trg_mask
        )
        out = self.fc_out(out)

        return out

In [14]:
# Setup the training phase
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
load_model = False
save_model = True

# Training hyperparameters
num_epochs = 5
learning_rate = 1e-4
batch_size = 32

# Model hyperparameters
src_vocab_size = len(korean.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 6  # in the paper 6
num_decoder_layers = 6
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi['<pad>']

# Tensorboard for nice plots
writer = SummaryWriter('runs/loss_plot')
step = 0

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = batch_size,
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device
)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = english.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [16]:
def translate_sentence(model, sentence, korean, english, device, max_length=50):
    
    tokens = mecab.morphs(sentence)
    # tokens = komoran.morphs(sentence)
    # tokens = tokenize_khaiii(sentence)

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, korean.init_token)
    tokens.append(korean.eos_token)

    # Go through each korean token and convert to an index
    text_to_indices = [korean.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]

# train the model

In [17]:
sentence = '이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다.'  # df['kor'][117]

for epoch in range(num_epochs):

    model.eval()
    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # forward prop
        output = model(inp_data, target[:-1])
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        optimizer.zero_grad()

        loss = criterion(output, target)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        
        writer.add_scalar('Training loss', loss, global_step=step)
        step += 1


    translated_sentence = translate_sentence(model, sentence, korean, english, device, max_length = 100)
    translated_sentence = ' '.join(translated_sentence)
    translated_sentence = translated_sentence.replace(' ,', ',')
    translated_sentence = translated_sentence.replace(' .', '.')
    translated_sentence = translated_sentence.replace(' <eos>', '')

    print(f'[Epoch] {epoch+1} / {num_epochs}')
    print(f'[Loss] {loss:.4f}')
    print(f'[Exsample] {sentence} >>> {translated_sentence}')

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


[Epoch] 1 / 5
[Loss] 5.5955
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> the government is the <unk> is the government of the <unk> is the <unk>.
[Epoch] 2 / 5
[Loss] 5.7968
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> this is because the most are the world of the <unk> of the year.
[Epoch] 3 / 5
[Loss] 5.0973
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> many people are many many many people in the <unk> of the <unk>.
[Epoch] 4 / 5
[Loss] 4.6091
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> there are many many people in the people of the <unk> of the <unk>.
[Epoch] 5 / 5
[Loss] 4.9534
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> many people are many of the country of the <unk> are many.


# test the model

In [18]:
from torchtext.data.metrics import bleu_score

def bleu(data, model, korean, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, korean, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

bleu(test_data, model, korean, english, device)

NotImplementedError: ignored

# look into the data structure

How to check the data structure?
- vars(data)
- data.\__dict__

In [19]:
vars()  # 변수 확인 함수

{'BucketIterator': torchtext.data.iterator.BucketIterator,
 'Field': torchtext.data.field.Field,
 'In': ['',
  "get_ipython().system('nvidia-smi')",
  "from psutil import virtual_memory\nram_gb = virtual_memory().total / 1e9\nprint('Your runtime has {:.1f} gigabytes of available RAM\\n'.format(ram_gb))",
  "import time\nstart = time.time()\n\nget_ipython().system('pip install konlpy')\nget_ipython().system('sudo apt-get install curl git')\nget_ipython().system('bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)')\n\nprint(f'mecab 설치 소요 시간: {((time.time()-start)/60):.2f} min')",
  "start = time.time()\n\nget_ipython().system('python -m spacy download en')\n\nprint(f'spacy tokenizer(en) 설치 소요 시간: {((time.time()-start)/60):.2f} min')",
  "from konlpy.tag import Mecab\nimport spacy\n\nmecab = Mecab()\nspacy_en = spacy.load('en')\n\ndef tokenize_en(text):\n    return [tok.text for tok in spacy_en.tokenizer(text)]\n\nprint('tokenization test with sample t

In [20]:
test_data

In [ ]:
vars(test_data)

In [ ]:
test_data.__dict__ 

In [24]:
len(test_data)

6000

In [25]:
test_data[0]

In [26]:
print(vars(test_data[0]))

{'src': ['중', '장기', '적', '으로', 'cj', 'enm', '의', '기업', '가치', '가', '상승', '할', '것', '이', '란', '전망', '을', '내놓', '고', '있', '다', '.'], 'trg': ['it', 'is', 'predicting', 'that', 'cj', 'enm', "'s", 'corporate', 'value', 'will', 'rise', 'in', 'the', 'mid-', 'to', 'long', '-', 'term', '.']}


In [27]:
print(test_data[0].__dict__)

{'src': ['중', '장기', '적', '으로', 'cj', 'enm', '의', '기업', '가치', '가', '상승', '할', '것', '이', '란', '전망', '을', '내놓', '고', '있', '다', '.'], 'trg': ['it', 'is', 'predicting', 'that', 'cj', 'enm', "'s", 'corporate', 'value', 'will', 'rise', 'in', 'the', 'mid-', 'to', 'long', '-', 'term', '.']}


- source sentence (korean)

In [28]:
src = vars(test_data[0])["src"]

print(src)

['중', '장기', '적', '으로', 'cj', 'enm', '의', '기업', '가치', '가', '상승', '할', '것', '이', '란', '전망', '을', '내놓', '고', '있', '다', '.']


- target sentence (english)

In [29]:
trg = vars(test_data[0])["trg"]

print(trg)

['it', 'is', 'predicting', 'that', 'cj', 'enm', "'s", 'corporate', 'value', 'will', 'rise', 'in', 'the', 'mid-', 'to', 'long', '-', 'term', '.']


# test the translate_sentence() function
- 입력 문장을 list에서 string으로 변경

In [32]:
src_txt = ' '.join(src)
src_txt = src_txt.replace(' .', '.')
src_txt

'중 장기 적 으로 cj enm 의 기업 가치 가 상승 할 것 이 란 전망 을 내놓 고 있 다.'

In [35]:
prediction = translate_sentence(model, src_txt, korean, english, device, max_length = 100)
prediction = prediction[:-1]

print(prediction)

['the', 'government', 'is', 'expected', 'to', 'be', 'the', 'company', 'to', 'be', 'expected', 'to', 'be', 'expected', 'to', 'the', 'future', 'of', 'the', 'future', '.']


# BLEU score test

In [37]:
# bleu_score() function example

candidate_corpus = [['My', 'full', 'pytorch', 'test'], ['Another', 'Sentence']]
references_corpus = [[['My', 'full', 'pytorch', 'test'], ['Completely', 'Different']], [['No', 'Match']]]
bleu_score(candidate_corpus, references_corpus)

0.8408964276313782

In [42]:
targets = []
outputs = []

targets.append([trg])
outputs.append(prediction)

In [43]:
print(targets)
print(outputs)

[[['it', 'is', 'predicting', 'that', 'cj', 'enm', "'s", 'corporate', 'value', 'will', 'rise', 'in', 'the', 'mid-', 'to', 'long', '-', 'term', '.']]]
[['the', 'government', 'is', 'expected', 'to', 'be', 'the', 'company', 'to', 'be', 'expected', 'to', 'be', 'expected', 'to', 'the', 'future', 'of', 'the', 'future', '.']]


In [44]:
bleu_score(outputs, targets)

0.0

# Test the model again

In [45]:
from torchtext.data.metrics import bleu_score

def bleu(data, model, korean, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        src = ' '.join(src).replace(' .', '.')
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, korean, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

bleu(test_data, model, korean, english, device)

0.018196161836385727

# Conclusion

**translate_sentence()에서 에러 발생한 이유**
- model training code에서 에러가 나지 않았던 이유는 입력 sentence가 토큰화된 리스트가 아닌, string 문자형이였기 때문
- model test에서 사용되었던 bleu()함수에는 translate_sentence()함수가 들어있었는데, 입력 sentence가 토큰화된 리스트로 입력되어서 에러가 발생하였음
- 독영병렬코퍼스로 학습을 시켰을 경우 list->list 번역이 가능했던 반면, 한영병렬코퍼스는 string->list 번역만 가능하였음
- 어떤 차이점이 있는지 translate_sentence()함수를 분석해볼 필요가 있음